## 데이터 : 개/고양이 binary classification
## 모델 : ResNet-14¶
## 평가 사항 : Image Augmentation 
### 약 2,000개 → 약 40,000개로 이미지 데이터 확장
## 평가 결과 : accuracy 상승 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;0.65 ~ 0.7 → 0.8 ~ 0.85


# <font color='red'>Convolutional Neural Network </font>
- 작성자 : [Gauss Kim](https://github.com/gaussian37)<br>
- 목적 : CNN의 다양한 model을 Custom Data에 맞게 사용할 수 있도록 지원<br>

In [1]:
import os
import numpy as np
from scipy.misc import imread, imresize, imshow
import tensorflow as tf
import matplotlib.pyplot as plt
import random

#  <font color='blue'>Hyper parameter</font>#

### - Image 사이즈 및 갯수 ###
- resize할 image의 height와 width에 관한 hyper parameter
- 이미지 갯수

In [2]:
img_height = 64 # 이미지 height 크기
img_width = 64 # 이미지 width 크기
n_train = 30324 # train 이미지 갯수
n_val = 7582# val 이미지 갯수
n_test = 9477 # test 이미지 갯수
n_class = 2 # class(카테고리) 갯수
is_preproc = True # Input normalization 사용 여부

### - random seed ###
랜덤 변수 시드 설정

In [3]:
seed = 777
tf.set_random_seed(seed)

### - Learning rate 초깃값 ###
Back propagation 시 parameter update에 사용

In [4]:
learning_rate = 0.01

### - Training Epoch ###

In [5]:
training_epochs = 30

### - batch 사이즈 ###

In [6]:
batch_size = 32

# <font color='blue'>TFRecord 읽어 오기</font> #

### - tfrecord 파일명 및 경로 설정 ###

In [7]:
tfrecord_train = 'train.tfrecord' # train 데이터 tfrecord
tfrecord_val = 'val.tfrecord' # validation 데이터 tfrecord
tfrecord_test = 'test.tfrecord' # test 데이터 tfrecord
tfrecord_dir = 'tfrecords' # tfrecord 폴더

### - 이미지와 라벨 읽기 함수 생성 ###

In [8]:
# 이미지와 라벨 읽어오기
def read_and_decode(filename_queue, n_batch):    
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    
    features = tf.parse_single_example(
        serialized_example,
        features={            
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        })    
    # Convert from a scalar string tensor
    image = tf.decode_raw(features['image'], tf.uint8)        
    label = tf.cast(features['label'], tf.int32)
    label_onehot = tf.one_hot(label, depth=n_class)
    
    image = tf.reshape(image, [img_height, img_width, 3])    
    
    images, labels = tf.train.batch([image, label_onehot],
                                           batch_size=n_batch,
                                           capacity=10000,
                                           num_threads=4)    
    return images, labels 

### - image, label 파일 인풋 생성

In [9]:
# **** File input ***
cwd = os.getcwd()
train_path = os.path.join(cwd, tfrecord_dir, tfrecord_train)
val_path = os.path.join(cwd, tfrecord_dir, tfrecord_val)
test_path = os.path.join(cwd, tfrecord_dir, tfrecord_test)

filename_queue_train = tf.train.string_input_producer([train_path], num_epochs=training_epochs)
image_batch, label_batch = read_and_decode(filename_queue_train, batch_size)

filename_queue_val = tf.train.string_input_producer([val_path], num_epochs=training_epochs)
image_val, label_val = read_and_decode(filename_queue_val, batch_size)

filename_queue_test = tf.train.string_input_producer([test_path], num_epochs=1)
image_test, label_test = read_and_decode(filename_queue_test, n_test)

# <font color='blue'>Function for Optimizing Neural Network</font>

In [10]:
# 입력값 normalization : 이미지의 평균값 만큼 전체 이미지 값 감소
def preproc(x):    
    mean = tf.reduce_mean(x, axis=1, keep_dims=True)
    return x - mean

# <span style="background-color: #FFFF00"> <font color='red'>ResNet</font></span> #

### Hyper parameter ###

In [11]:
number_of_resblk = 4
number_of_layer = 3

In [12]:
def conv_bn_activ(name, x, n_filters, kernel_size, strides, training, seed, padding='SAME'):
    with tf.variable_scope(name):
        net = tf.layers.conv2d(x, n_filters, kernel_size, strides=strides, padding=padding, use_bias=False, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
        net = tf.layers.batch_normalization(net, training=training)
        net = tf.nn.relu(net)
    return net

In [13]:
def residual_block(name, x, n_filters, training, seed, downsample=False):    
    if downsample:
        strides = 2
    else:
        strides = 1
    with tf.variable_scope(name):
        net1 = conv_bn_activ("inner_conv1",x, n_filters, [3, 3], strides, training, seed)
        net2 = conv_bn_activ("inner_conv2",x, n_filters, [3, 3], strides, training, seed)
        
        if downsample:
            x = tf.layers.conv2d(x, n_filters, [1, 1], strides=2, padding='SAME',kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
        return tf.nn.relu(net2 + x)

In [14]:
def build_resnet(X_img, resblk_n, layer_n, n_filters, training, seed):
    net = X_img    
    with tf.variable_scope("CONV0"):
        net = conv_bn_activ("pre_conv", net, n_filters, [3, 3], 1, training, seed)
        print(net)
    
    # 블락 생성
    for i in range(resblk_n):
        with tf.variable_scope("CONV"+str(i+1)):
            for j in range(layer_n):                
                net = residual_block("resblk{}".format(j), net, n_filters, training, seed, (j==0))
                print(net)
            n_filters *= 2
            
    with tf.variable_scope("GAP"):        
        pool_height, pool_width = net.shape[1:3]        
        net = tf.layers.average_pooling2d(name="gap", inputs=net, pool_size=[pool_height, pool_width], strides=1, padding='VALID')
        print(net)
        
    with tf.variable_scope("FC"):
        logits = tf.layers.dense(net, n_class, name="logits", kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
        logits = tf.reshape(logits, [-1, n_class]) # (?, 1, 1, n_class) → (?, n_class)
        print(logits)
    return logits

# <font color='blue'>Neural Network Design</font>#

### - Input Design ###

In [15]:
X = tf.placeholder(tf.float32, [None, img_height, img_width, 3], name="X")
Y = tf.placeholder(tf.float32, [None, n_class], name = "Y")
is_train = tf.placeholder(tf.bool, name="is_train")

#if is_preproc:
#    X = preproc(X)

In [16]:
logits = build_resnet(X, number_of_resblk, number_of_layer, training=is_train, n_filters=16, seed=seed)

Tensor("CONV0/pre_conv/Relu:0", shape=(?, 64, 64, 16), dtype=float32)
Tensor("CONV1/resblk0/Relu:0", shape=(?, 32, 32, 16), dtype=float32)
Tensor("CONV1/resblk1/Relu:0", shape=(?, 32, 32, 16), dtype=float32)
Tensor("CONV1/resblk2/Relu:0", shape=(?, 32, 32, 16), dtype=float32)
Tensor("CONV2/resblk0/Relu:0", shape=(?, 16, 16, 32), dtype=float32)
Tensor("CONV2/resblk1/Relu:0", shape=(?, 16, 16, 32), dtype=float32)
Tensor("CONV2/resblk2/Relu:0", shape=(?, 16, 16, 32), dtype=float32)
Tensor("CONV3/resblk0/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("CONV3/resblk1/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("CONV3/resblk2/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("CONV4/resblk0/Relu:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("CONV4/resblk1/Relu:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("CONV4/resblk2/Relu:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("GAP/gap/AvgPool:0", shape=(?, 1, 1, 128), dtype=float32)
Tensor("FC/Reshape:0", shape=(?, 2), dtype=flo

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="cost")
#n_batches_per_epoch = int(n_train / batch_size)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, name="optimizer") 

In [18]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(logits.shape, Y.shape)

# pred = tf.argmax(logits, axis=1, name="prediction")
# prob = tf.nn.softmax(logits, name="softmax")
# accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(Y, axis=1)), tf.float32), name="accuracy")

(?, 2) (?, 2)


In [19]:
train_var = [X, Y, is_train, logits, accuracy]
tf.add_to_collection('train_var', train_var[0])
tf.add_to_collection('train_var', train_var[1])
tf.add_to_collection('train_var', train_var[2])
tf.add_to_collection('train_var', train_var[3])
tf.add_to_collection('train_var', train_var[4])
saver = tf.train.Saver()

In [20]:
# initialize
init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(tf.global_variables_initializer())
sess.run(init_op)

In [21]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

In [22]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0.
    avg_train_acc = 0.
    avg_val_acc = 0.
    total_batch = int(n_train / batch_size)
    total_batch_val = int(n_val / batch_size)

    for i in range(total_batch):
        try:
            batch_xs, batch_ys = sess.run([image_batch, label_batch])
        except tf.errors.OutOfRangeError:
            pass
        batch_xs = batch_xs/255.        
        feed_dict = {X: batch_xs, Y: batch_ys, is_train: True}
        #feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        acc, c, _ = sess.run([accuracy, cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_train_acc += acc / total_batch
    
    for i in range(total_batch_val):
        batch_xs, batch_ys = sess.run([image_val, label_val])
        batch_xs = batch_xs/255.
        feed_dict = {X: batch_xs, Y: batch_ys, is_train: False}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        avg_val_acc += acc / total_batch_val

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'train accuracy = ', 
         '{:.5f}'.format(avg_train_acc), 'validation accuracy = ', '{:.5f}'.format(avg_val_acc))


print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 0001 cost = 0.693980999 train accuracy =  0.61609 validation accuracy =  0.62156
Epoch: 0002 cost = 0.582419753 train accuracy =  0.69598 validation accuracy =  0.67307
Epoch: 0003 cost = 0.544961508 train accuracy =  0.72528 validation accuracy =  0.68829
Epoch: 0004 cost = 0.510449974 train accuracy =  0.74960 validation accuracy =  0.73517
Epoch: 0005 cost = 0.481077368 train accuracy =  0.76907 validation accuracy =  0.77132
Epoch: 0006 cost = 0.450612768 train accuracy =  0.78881 validation accuracy =  0.75847
Epoch: 0007 cost = 0.420703082 train accuracy =  0.80511 validation accuracy =  0.74351
Epoch: 0008 cost = 0.394495651 train accuracy =  0.82237 validation accuracy =  0.77635
Epoch: 0009 cost = 0.372262270 train accuracy =  0.83121 validation accuracy =  0.77225
Epoch: 0010 cost = 0.348893664 train accuracy =  0.84599 validation accuracy =  0.79251
Epoch: 0011 cost = 0.325244992 train accuracy =  0.85698 validation accuracy =  0.8

In [24]:
test_xs, test_ys = sess.run([image_test, label_test])
print('Test Accuracy:', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys, is_train:False}))

OutOfRangeError: FIFOQueue '_6_batch_2/fifo_queue' is closed and has insufficient elements (requested 9477, current size 0)
	 [[Node: batch_2 = QueueDequeueManyV2[component_types=[DT_UINT8, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](batch_2/fifo_queue, batch_2/n)]]

Caused by op 'batch_2', defined at:
  File "C:\Users\infoe\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\infoe\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\infoe\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\infoe\Anaconda3\lib\site-packages\traitlets\config\application.py", line 653, in launch_instance
    app.start()
  File "C:\Users\infoe\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\infoe\Anaconda3\lib\site-packages\zmq\eventloop\i